In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.pipeline import  Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler , OrdinalEncoder ,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split ,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression ,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression
from sklearn.svm import SVC

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:

svc=SVC()
xgbc=XGBClassifier()
knc=KNeighborsClassifier()
rfc=RandomForestClassifier()
lr=LogisticRegression()

best_svc=RandomizedSearchCV(svc,{'C':[0.5,1,1.5,2],'kernel':['poly','linear'],'degree':[2,3,4,5,8]},cv=10,n_iter=30)

best_knc=RandomizedSearchCV(knc,{'n_neighbors': np.arange(1, 25)},cv=10,n_iter=30)


best_rfc=RandomizedSearchCV(rfc,{'n_estimators':[100,150,170,200,250,300,500,600,700,1000],
                                 
                                  'max_depth':[5,6,7,8,9,10,11,12,13,14,20],
                                  'criterion':['gini', 'entropy']},cv=10,n_iter=30)

best_lr=RandomizedSearchCV(lr,{
                                'penalty':['l1','l2', 'elasticnet', 'none'],
                                'C':[0.01,0.1,1,1.5,2,3],
                                'max_iter':[10,100,1000,2000],
                                'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},cv=10,n_iter=30)

best_xgbc=RandomizedSearchCV(xgbc,{
    'max_depth':[6,7,8,9,10,11,15,20,30] ,
    'min_child_weight': [1,2,4,5,7,8,9] ,
    'eta':[0.1,0.5,0.7,0.01,1,2] ,
    'subsample': [0.1,0.5,0.01,1,2] ,
    'colsample_bytree': [0.1,0.5,0.7,0.01,1,2]},cv=10,n_iter=30)


In [3]:

data_train=pd.read_csv('/kaggle/input/titanic/train.csv',index_col='PassengerId')
data_test=pd.read_csv('/kaggle/input/titanic/test.csv',index_col='PassengerId')


train_y=data_train.Survived    
train_x=data_train.drop(columns=['Ticket','Name','Cabin','Survived' ])
test_x=data_test.drop(columns=['Name','Ticket','Cabin'])

cat_col=train_x.select_dtypes('object').columns.tolist()
num_col=train_x.select_dtypes(['float','int']).columns.tolist()

train_x=train_x[num_col+cat_col]
test_x=test_x[num_col+cat_col]


In [4]:

pipline_num=Pipeline([('imputer_num',SimpleImputer(strategy='mean'))])
pipline_cat=Pipeline([('imputer_cat', SimpleImputer(strategy='most_frequent')),('encoder',OrdinalEncoder())])
full_pipline=ColumnTransformer([('num',pipline_num,num_col),('cat',pipline_cat,cat_col)])
train_x_final=pd.DataFrame(full_pipline.fit_transform(train_x),index=train_x.index,columns=train_x.columns)
test_x_final=pd.DataFrame(full_pipline.fit_transform(test_x),index=test_x.index,columns=train_x.columns)
ind=test_x_final.index

In [5]:

best_models=[best_xgbc,best_knc, best_rfc,best_lr,best_svc]
max=0
for best in  best_models :
    best.fit(train_x_final,train_y)
    new_model=best.best_estimator_
    if best.best_score_>max :
        max=best.best_score_
        best_model=new_model
    print('best pra',best.best_params_)
    print('new model :',new_model)
    print(best.best_score_)
print('best model is : ',best_model)
        




/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:10:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/xgboost/core.py", line 436, in inner_f
    return f(**kwargs)
  File "/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py", line 1187, in fit
    callbacks=callbacks,
  File "/opt/conda/lib/python3.7/site-packages/xgboost/training.py", line 197, in train
    early_stopping_rounds=early_stopping_rounds)
  File "/opt/conda/lib/python3.7/site-packages/xgboost/training.py", line 81, in _train_internal
    bst.update(dtrain, i, obj)
  File "/opt/conda/lib/python3.7/site-packages/xgboost/core.py", line 1501, in update
    dtrain.hand

[17:11:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:11:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:11:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:11:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 24 is smaller than n_iter=30. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


best pra {'n_neighbors': 3}
new model : KNeighborsClassifier(n_neighbors=3)
0.7172409488139826
best pra {'n_estimators': 300, 'max_depth': 8, 'criterion': 'gini'}
new model : RandomForestClassifier(max_depth=8, n_estimators=300)
0.8373033707865168


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio para

best pra {'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 2000, 'C': 0.1}
new model : LogisticRegression(C=0.1, max_iter=2000)
0.7980149812734083
best pra {'kernel': 'linear', 'degree': 3, 'C': 1.5}
new model : SVC(C=1.5, kernel='linear')
0.786729088639201
best model is :  RandomForestClassifier(max_depth=8, n_estimators=300)


In [6]:
predictions=pd.DataFrame(best_model.predict(test_x_final),columns=['Survived'],  index=test_x_final.index)
print(predictions)
predictions.to_csv('predictions3.csv')


             Survived
PassengerId          
892                 0
893                 0
894                 0
895                 0
896                 1
...               ...
1305                0
1306                1
1307                0
1308                0
1309                0

[418 rows x 1 columns]
